In [1]:
from dask.distributed import Client, progress

from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=40)
cluster

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:  tcp://10.32.16.125:34685
distributed.scheduler - INFO -   dashboard at:                     :8787


In [2]:
## path for mdules
import sys
sys.path.insert(0,"/home/jovyan/xscale")
import xscale


## imports

import numpy as np
import numpy.ma as ma
import xarray as xr
import time
import pandas as pd

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.cm as mplcm
seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

### quick plot
import matplotlib.pyplot as plt

import glob
import os 

%matplotlib inline

In [3]:
import gcsfs
fs = gcsfs.GCSFileSystem('pangeo-181919', requester_pays=True)


In [4]:
zmap_grid = fs.get_mapper('pangeo-meom/eNATL60-I/eNATL60-mesh-mask')
ds_grid = xr.open_zarr(zmap_grid)


In [5]:
zmap_u = fs.get_mapper('pangeo-meom/eNATL60-BLBT02-SSU-1h')
ds_u = xr.open_zarr(zmap_u)


In [6]:
zmap_v = fs.get_mapper('pangeo-meom/eNATL60-BLBT02-SSV-1h')
ds_v = xr.open_zarr(zmap_v)


In [7]:
navlat= ds_grid['nav_lat']
navlon= ds_grid['nav_lon']
e1u=ds_grid.e1u[0]
e1v=ds_grid.e1v[0]
e2u=ds_grid.e2u[0]
e2v=ds_grid.e2v[0]
ff=ds_grid.ff[0]
fmask=ds_grid.fmask[0,0]

In [8]:
def filt(w):
    win_box2D = w.window
    win_box2D.set(window='hanning', cutoff=90, dim=['x', 'y'], n=[90, 90])
    bw = win_box2D.boundary_weights(drop_dims=[])
    w_LS = win_box2D.convolve(weights=bw)
    w_SS=w-w_LS
    return w_SS


In [9]:
def curl(u,v,e1v,e2u,ff):
    '''
    This routine computes the relative vorticity from 2D fields of horizontal velocities and the spatial Coriolis parameter.
    '''
    #Computation of dy(u)
    fe2u=1/e2u
    fse2u=fe2u.squeeze()
    dyu=(u.shift(y=-1) - u)*fse2u
    #Computation of dx(v)
    fe1v=1/e1v
    fse1v=fe1v.squeeze()
    dxv=(v.shift(x=-1) - v)*fse1v
    #Projection on the grid T
    dxvt=0.5*(dxv.shift(y=-1)+dxv)
    dyut=0.5*(dyu.shift(x=-1)+dyu)
    #Computation of the vorticity divided by f
    fff=1/ff
    curl=(dxvt-dyut)*fff
    return curl

In [10]:
def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """
    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)
    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()
    A1.coords[dim] = ind
    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)
    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)
    for d in newdims[::-1]:
        dims.insert(i, d)
    for d in newdims:
        _ = dims.pop(-1)
    return A1.transpose(*dims)

def boxcar_reshape(array2D,icrs,jcrs,isize,jsize):
    """Return a 3D array where values in boxes added in extra dimensions 
    """
    jpj, jpi = array2D.shape
    if jpj%jcrs==0 and jpi%icrs==0:
        t=xr_reshape(array2D,'x',['xcrs','icrs'],[np.arange(isize),np.arange(icrs)])
        tt=xr_reshape(t,'y',['ycrs','jcrs'],[np.arange(jsize),np.arange(jcrs)]) 
        ttt=tt.stack(ijcrs=('jcrs','icrs'))
        return ttt
    else:
        print("shape and coarsening factors are not compatible")
        return

def coarse_var(var):
    jpj,jpi = var.shape
    x_offset=0
    y_offset=0
    crs_factor=60
    jcrs, icrs = crs_factor, crs_factor
    jsize = jpj - (jpj - y_offset) % jcrs
    isize = jpi - (jpi - x_offset) % icrs
    inb=(jpi - x_offset) // icrs
    jnb=(jpj - y_offset) // jcrs
    cut_array = lambda array2D:array2D[...,y_offset:jsize,x_offset:isize]
    var_cut=cut_array(var.squeeze())
    var_rechunk=var_cut.chunk({'y':600,'x':600})
    varcrs=boxcar_reshape(var_rechunk,crs_factor,crs_factor,inb,jnb)
    varcrsm=varcrs.mean(dim='ijcrs')
    return varcrsm

In [11]:
def plot_fine_scale_variance(var,loncrs,latcrs,lon,lat,fmask,month,m,config,case):
    ''' map of the averaged fine scale variance
    '''
    fig, ax = plt.subplots(1,1,figsize=(15,10))
    ax = plt.subplot(111,projection=ccrs.PlateCarree(central_longitude=0))
    ax.autoscale(tight=True)
    gl = ax.gridlines(draw_labels=True, linestyle=':', color='black',
                      alpha=0.5)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 25, 'color': 'gray'}
    gl.ylabel_style = {'size': 25, 'color': 'gray'}
    
    ax.tick_params('both',labelsize=25)

    pcolor = ax.pcolormesh(loncrs,latcrs,ma.masked_invalid(var),cmap=seq_cmap,vmin=0,vmax=0.1,alpha=1)
    ax.contour(lon,lat,fmask,alpha=0.5,linewidth=0.000001,antialiased=True,colors='black')
    cbar = plt.colorbar(pcolor,orientation='horizontal',pad=0.1)
    cbar.ax.tick_params(labelsize=25)
    cbar.set_label('Small scales surface vorticity variance in '+month+' for '+config+'-'+case,fontsize=15)
    plt.savefig(config+'-'+case+'_m'+m+'_fine_scale_variance_vorticity.png')
  

In [12]:
print('Select dates in zarr')
u=ds_u.sel(time_counter=slice('2010-03-01','2010-03-31'))['sozocrtx']
v=ds_v.sel(time_counter=slice('2010-03-01','2010-03-31'))['somecrty']


Select dates in zarr


In [13]:
chunkt=1
chunkx=4000
chunky=np.int(np.floor(125*(1000**2)/(chunkt*chunkx*u[0].dtype.itemsize)))
u_rechunk=u.chunk({'time_counter':chunkt,'y':chunky,'x':chunkx})
v_rechunk=v.chunk({'time_counter':chunkt,'y':chunky,'x':chunkx})

In [14]:
print('Computing curl')
curl_surf=curl(u_rechunk,v_rechunk,e1v,e2u,ff)
print('Filtering curl')
curl_SS=filt(curl_surf)
curl_LS=curl_surf-curl_SS
hpcurl=curl_SS
print('**2 and temp mean')
hpcurl2 = hpcurl ** 2
hpcurl2m = hpcurl2.mean(axis=0,keep_attrs=True)


Computing curl
Filtering curl
**2 and temp mean


In [15]:
print('Coarsening of the grid')
latcrsm=coarse_var(navlat)
loncrsm=coarse_var(navlon)
      

Coarsening of the grid


In [16]:
hpcurl2mcm = coarse_var(hpcurl2m)


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, these coordinates will be transposed as well unless you specify transpose_coords=False.
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, these coordinates will be transposed as well unless you specify transpose_coords=False.


In [ ]:
hpcurl2mcm.values

distributed.scheduler - INFO - Register tcp://10.32.65.3:42379
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.65.3:42379
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.65.2:37075
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.65.2:37075
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.9.3:37819
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.9.3:37819
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.9.2:39305
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.9.2:39305
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.32.10.3:36957
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.10.3:36957
distributed.core - INFO - S

In [ ]:
print('Plotting')
plot_fine_scale_variance(hpcurl2mcm,loncrsm, latcrsm,navlon,navlat,fmask,'March',3,'eNATL60','BLBT02')


Plotting


In [ ]:
print('Select dates in zarr')
u=ds_u.sel(time_counter=slice('2009-09-01','2009-09-30'))['sozocrtx']
v=ds_v.sel(time_counter=slice('2009-09-01','2009-09-30'))['somecrty']
u_rechunk=u.chunk({'time_counter':1,'y':4729,'x':8354})
v_rechunk=v.chunk({'time_counter':1,'y':4729,'x':8354})
print('Computing curl')
curl_surf=curl(u_rechunk,v_rechunk,e1v,e2u,ff)
print('Filtering curl')
curl_SS=filt(curl_surf)
curl_LS=curl_surf-curl_SS
hpcurl=curl_SS
print('**2 and temp mean')
hpcurl2 = hpcurl ** 2
hpcurl2m = hpcurl2.mean(axis=0,keep_attrs=True)
print('Coarsening of the grid')
print('Coarsening of the grid')
hpcurl2mcm = coarse_var(hpcurl2m)
latcrsm=coarse_var(navlat)
loncrsm=coarse_var(navlon)
print('Plotting')
plot_fine_scale_variance(hpcurl2mcm,loncrsm, latcrsm,navlon,navlat,fmask,'September',month,config,case)
      